# Currently test a way of getting historical projections to compare to my models

In [1]:
import pandas as pd
import unicodedata

In [14]:
actual = pd.read_csv('../Resources/all_data.csv')
actual = actual[['IDfg','Season', 'Name', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G', 'AB', 'PA', 'AVG', 'BB%', 'OPS', 'ISO', 'wRAA', 'wRC+']]
actual = actual[(actual['Season'] >= 2010) & (actual['Season'] <= 2024)]
actual['IDfg'] = actual['IDfg'].astype('object')

C:\Users\thoma\AppData\Local\Temp\ipykernel_33780\1461721172.py:1: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  actual = pd.read_csv('../Resources/all_data.csv')


In [4]:
file_path = f"../Resources/steamer_preseason_projections/all_steamer_data.csv"
df = pd.read_csv(file_path)

In [ ]:
columns_to_keep = ['IDfg','Season', 'Name', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G', 'AB', 'PA', 'AVG', 'BB%', 'OPS', 'ISO', 'wRAA', 'wRC+']
df = df[[col for col in columns_to_keep if col in df.columns]]


In [6]:
df = df.rename(columns={col: f'Projected_{col}' for col in df.columns if col not in ['IDfg','Name', 'Season']})

In [15]:
steamer_df = pd.merge(df, actual, on=['IDfg', 'Season'], how='inner')

In [16]:
steamer_df

,IDfg,Season,Name_x,Projected_HR,Projected_R,Projected_RBI,Projected_SB,Projected_WAR,Projected_G,Projected_PA,...,WAR,G,AB,PA,AVG,BB%,OPS,ISO,wRAA,wRC+


In [125]:
# List of years
years = list(range(2012, 2025))

# List to store DataFrames
dfs = []

# Define the columns you want to keep
columns_to_keep = ['Season', 'Name', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G', 'AB', 'PA', 'AVG', 'BB%', 'OPS', 'ISO', 'wRAA', 'wRC+']


for year in years:
    file_path = f"../Resources/steamer_preseason_projections/steamer_projections_preseason_{year}.csv"
    df = pd.read_csv(file_path)

    # Ensure only the specified columns are kept (if they exist in the dataset)
    df = df[[col for col in columns_to_keep if col in df.columns]]

    # Add the 'Season' column
    df['Season'] = year  

    # Rename stat columns to "Projected_{stat}"
    df = df.rename(columns={col: f'Projected_{col}' for col in df.columns if col not in ['Name', 'Season']})

    # Reorder columns to ensure 'Season' comes right after 'Name'
    column_order = ['Name', 'Season'] + [col for col in df.columns if col not in ['Name', 'Season']]
    df = df[column_order]

    dfs.append(df)

# Concatenate all DataFrames into one
steamer_combined = pd.concat(dfs, ignore_index=True)



In [126]:
def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

In [127]:
steamer_combined = steamer_combined[(steamer_combined['Projected_PA'] > 10)]
steamer_combined = steamer_combined.drop_duplicates(subset=['Name', 'Season'])

In [128]:
steamer_combined['Name'] = steamer_combined['Name'].apply(remove_accents)
actual['Name'] = actual['Name'].apply(remove_accents)

In [130]:
steamer_df = pd.merge(steamer_combined, actual, on=['Name', 'Season'], how='inner')
steamer_df.to_csv('../Projection_Results/steamer.csv', index=False)

In [131]:
steamer_df.shape

(6748, 44)

In [136]:
# List of years
years = list(range(2010, 2025))

# List to store DataFrames
dfs = []

# Define the columns you want to keep
columns_to_keep = ['Season', 'Name', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'wRC', 'WAR', 'G', 'AB', 'PA', 'AVG', 'BB%', 'OPS', 'ISO', 'wRAA', 'wRC+']


for year in years:
    file_path = f"../Resources/zips_preseason_projections/zips_projections_preseason_{year}.csv"
    df = pd.read_csv(file_path)

    # Ensure only the specified columns are kept (if they exist in the dataset)
    df = df[[col for col in columns_to_keep if col in df.columns]]

    # Add the 'Season' column
    df['Season'] = year  

    # Rename stat columns to "Projected_{stat}"
    df = df.rename(columns={col: f'Projected_{col}' for col in df.columns if col not in ['Name', 'Season']})

    # Reorder columns to ensure 'Season' comes right after 'Name'
    column_order = ['Name', 'Season'] + [col for col in df.columns if col not in ['Name', 'Season']]
    df = df[column_order]

    dfs.append(df)

# Concatenate all DataFrames into one
zips_combined = pd.concat(dfs, ignore_index=True)

In [137]:
zips_combined = zips_combined[(zips_combined['Projected_PA'] > 10)]
zips_combined = zips_combined.drop_duplicates(subset=['Name', 'Season'])

In [142]:
zips_combined.shape

(18654, 23)

In [141]:
zips_combined = zips_combined.dropna(subset=['Name'])

In [143]:
zips_combined['Name'] = zips_combined['Name'].apply(remove_accents)

C:\Users\thoma\AppData\Local\Temp\ipykernel_18364\3553679501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips_combined['Name'] = zips_combined['Name'].apply(remove_accents)


In [144]:
zips_df = pd.merge(zips_combined, actual, on=['Name', 'Season'], how='inner')
zips_df.to_csv('../Projection_Results/zips.csv', index=False)